In [ ]:
# Install required packages
!pip install ta mplfinance yfinance joblib xgboost scikit-learn

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import joblib
import os
import requests
from datetime import datetime
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator
from ta.volatility import AverageTrueRange
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import mplfinance as mpf

In [ ]:
# Load F&O symbols
df_symbols = pd.read_csv("futures_list.csv")
futures_list = df_symbols["Symbol"].dropna().tolist()
print(f"Loaded {len(futures_list)} symbols for training and detection.")

In [ ]:
# Prepare dataset for training
records = []
for symbol in futures_list:
    df = yf.download(f"{symbol}.NS", period="1y", interval="1d", progress=False)
    if df.shape[0] < 20:
        continue

    # Ensure 1D series
    close = df["Close"]
    if isinstance(close, pd.DataFrame):
        close = close.squeeze()

    # Indicators
    df["RSI"] = RSIIndicator(close).rsi()
    macd = MACD(close)
    df["MACD_Hist"] = macd.macd_diff()

    high = df["High"]
    if isinstance(high, pd.DataFrame):
        high = high.squeeze()
    low = df["Low"]
    if isinstance(low, pd.DataFrame):
        low = low.squeeze()

    df["ADX"] = ADXIndicator(high, low, close).adx()

    # Fibonacci 0.618 retracement
    high_roll = high.rolling(10).max()
    low_roll = low.rolling(10).min()
    retrace = high_roll - (high_roll - low_roll) * 0.618
    df["near_618"] = (low <= retrace).astype(int)

    # Label: next-day return positive
    df["target"] = (close.shift(-1) > close).astype(int)
    df = df.dropna(subset=["RSI","MACD_Hist","ADX","near_618","target"])

    features = df[["RSI","MACD_Hist","ADX","near_618","Volume"]]
    for idx, row in features.iterrows():
        records.append({
            "RSI": row["RSI"],
            "MACD_Hist": row["MACD_Hist"],
            "ADX": row["ADX"],
            "near_618": row["near_618"],
            "Volume": row["Volume"],
            "target": int(df.loc[idx, "target"])
        })

train_df = pd.DataFrame(records)
print(f"Training dataset size: {train_df.shape}")

In [ ]:
# Train XGBoost model
X = train_df.drop("target", axis=1)
y = train_df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(
    objective="binary:logistic",
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)
model.fit(X_train, y_train)
print("Model trained. Test accuracy:", model.score(X_test, y_test))

# Save the model
joblib.dump(model, "fibonacci_xgb_model.pkl")
print("Saved model to fibonacci_xgb_model.pkl")

In [ ]:
# Detection & Alerts
model = joblib.load("fibonacci_xgb_model.pkl")

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")

def send_telegram(msg, chart_path=None):
    url = (
        f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto"
        if chart_path
        else f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    )
    data = (
        {'chat_id': TELEGRAM_CHAT_ID, 'caption': msg}
        if chart_path
        else {'chat_id': TELEGRAM_CHAT_ID, 'text': msg}
    )
    files = ({'photo': open(chart_path, 'rb')} if chart_path else None)
    requests.post(url, data=data, files=files)

today = datetime.today().strftime("%Y-%m-%d")
sent_file = "sent_signals.csv"
if not os.path.exists(sent_file):
    pd.DataFrame(columns=["date","symbol"]).to_csv(sent_file, index=False)
sent = pd.read_csv(sent_file)

results = []
for symbol in futures_list:
    df = yf.download(f"{symbol}.NS", period="6mo", interval="1d", progress=False)
    if df.empty:
        continue

    close = df["Close"]
    if isinstance(close, pd.DataFrame):
        close = close.squeeze()
    df["RSI"] = RSIIndicator(close).rsi()
    macd = MACD(close)
    df["MACD_Hist"] = macd.macd_diff()

    high = df["High"]
    if isinstance(high, pd.DataFrame):
        high = high.squeeze()
    low = df["Low"]
    if isinstance(low, pd.DataFrame):
        low = low.squeeze()
    df["ADX"] = ADXIndicator(high, low, close).adx()

    # Fibonacci
    high_roll = high.rolling(10).max()
    low_roll = low.rolling(10).min()
    retrace = high_roll - (high_roll - low_roll) * 0.618
    df["near_618"] = low <= retrace

    latest = df.dropna().iloc[-1]
    feats = np.array([
        latest["RSI"],
        latest["MACD_Hist"],
        latest["ADX"],
        int(latest["near_618"]),
        latest["Volume"]
    ]).reshape(1, -1)
    prob = model.predict_proba(feats)[0][1]

    if prob >= 0.7 and not ((sent["symbol"] == symbol) & (sent["date"] == today)).any():
        # ATR-based SL/TP
        atr = AverageTrueRange(high, low, close, window=14).average_true_range().iloc[-1]
        sl = float(close.iloc[-1] - atr)
        tp = float(close.iloc[-1] + 3 * atr)

        # Chart last 50 bars
        chart_df = df[-50:].copy()
        fib_lvl = float(retrace.iloc[-1])
        apds = [mpf.make_addplot([fib_lvl] * len(chart_df), type="line", linestyle="dashed")]

        mpf.plot(
            chart_df,
            type="candle",
            volume=True,
            addplot=apds,
            style="default",
            savefig="chart.png"
        )

        msg = (
            f"🚨 {symbol} Reversal Alert\n"
            f"Date: {today}\n"
            f"Prob: {prob:.1%}\n"
            f"RSI: {latest['RSI']:.1f}, MACD: {latest['MACD_Hist']:.2f}\n"
            f"SL: {sl:.2f}, TP: {tp:.2f}"
        )
        send_telegram(msg, chart_path="chart.png")

        sent = sent.append({"date": today, "symbol": symbol}, ignore_index=True)
        sent.to_csv(sent_file, index=False)
        results.append({"symbol": symbol, "prob": prob})

# Summary
df_results = pd.DataFrame(results)
print(df_results)